# Using WMLA Elastic Distributed Training via API - a sample notebook

<div class="alert alert-block alert-info">


### Contents

- [The big picture](#The-big-picture)
- [Changes to your code](#Changes-to-your-code)
- [Making dataset available](#Making-dataset-available)
- [Set up API end point and log on](#Set-up-API-end-point-and-log-on)
- [Submit job via API](#Submit-job-via-API)
- [Monitor running job](#Monitor-running-job)
- [Retrieve output and saved models](#Retrieve-output-and-saved-models)
  - [Output - Retrieve training output](#Output:--Retrieve-Training-Metric)
  - [Save Models](#Save-Model)
- [Debugging any issues](#Debugging-any-issues)
- [Further information and useful links](#Further-information-and-useful-links)
- [Appendix](#Appendix)


</div>

<div class="alert alert-block alert-danger">

Final things to do:
- add link to Learning Journey blog in sections: "The big picture" and "Changes to your code"
</div>

## The big picture
[Back to top](#Contents)

This notebook details the process of taking your existing model training code and making the changes required to run the code using [IBM Watson Machine Learning](https://developer.ibm.com/linuxonpower/deep-learning-powerai/powerai-enterprise/) (WMLA) using Elastic Distributed Training. 

<span style='color:deeppink'>**TODO:** Link to Learning Journey Blog</span>

The image below shows the various elements required to use EDT. In this notebook we will step through each of these elements in more detail. Through this process you will offload your code to a WMLA cluster, monitor the running job, retrieve the output and debug any issues seen. A [static version](https://github.com/IBM/wmla-assets/raw/master/WMLA-learning-journey/shared-images/5_running_job.png) is also available.

![overall](https://github.com/IBM/wmla-assets/raw/master/WMLA-learning-journey/shared-images/5_running_job.gif)

## Changes to your code
[Back to top](#Contents)

In this section we will take existing sample code and make the relevant changes required for use with EDT. An overview of these changes can be seen in the diagram below. A [static version](https://github.com/IBM/wmla-assets/raw/master/WMLA-learning-journey/shared-images/2_code_adaptations.png) is also available.

![code](https://github.com/IBM/wmla-assets/raw/master/WMLA-learning-journey/shared-images/2_code_adaptations.gif)



The key changes to your code in order to use EDT are the following:
- Importing libraries and setting up environment variables
- Data loading function for EDT
- Extract parameters for training
- Replace training and testing loops with EDT equivalents

For the purpose of this tutorial we have adapted the following RestNet18 model for use with EDT: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

Please see the blog associated with this notebook with more detailed explanation of the above changes.

<span style='color:deeppink'>**TODO:** Add link to Learning Journey blog in line above</span>

You can find the original code `pytorch_mnist.py` and the updated code `pytorch_mnist_EDT.py` in the zip file `pytorch_edt.tar.gz` contained in the [tutorial repository](https://github.com/IBM/wmla-assets/raw/master/WMLA-learning-journey/elastic-distributed-training-module/pytorch_edt.tar.gz).

You can run the following command to observe relative changes:
`diff -U4 pytorch_mnist.py pytorch_mnist_EDT.py`

Your modified code should be made available in a directory which also contains the EDT helper scripts: `edtcallback.py`, `emetrics.py` and `elog.py`. Sample versions can be found in the tarball in the tutorial repository; additionally they can be downloaded from http://ibm.biz/WMLA-samples.  

## Making dataset available
[Back to top](#Contents)

Next we will make our dataset available to the WMLA cluster as seen in the diagram below. 

![data](https://github.com/IBM/wmla-assets/raw/master/WMLA-learning-journey/shared-images/3_dataset.png)

1. Ask your system admin the path of $DLI_DATA_FS directory

2. ssh to WMLA-server and get access to $DLI_DATA_FS



3. Download dataset

```
[WMLA-server dlidata]# wget https://github.com/IBM/wmla-assets/raw/master/WMLA-learning-journey/elastic-distributed-training-module/pytorch-mnist-dataset.zip
--2020-05-12 22:27:07--  https://github.com/IBM/wmla-assets/raw/master/WMLA-learning-journey/elastic-distributed-training-module/pytorch-mnist-dataset.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/wmla-assets/master/WMLA-learning-journey/elastic-distributed-training-module/pytorch-mnist-dataset.zip [following]
--2020-05-12 22:27:07--  https://raw.githubusercontent.com/IBM/wmla-assets/master/WMLA-learning-journey/elastic-distributed-training-module/pytorch-mnist-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23006288 (22M) [application/zip]
Saving to: ‘pytorch-mnist-dataset.zip’

100%[=============================================================================================================================================>] 23,006,288  67.4MB/s   in 0.3s

2020-05-12 22:27:08 (67.4 MB/s) - ‘pytorch-mnist-dataset.zip’ saved [23006288/23006288]

[WMLA-server dlidata]# unzip pytorch-mnist-dataset.zip

```

4. Unzip the zip file and modify file owner/group, that is equivalent to Instance Group Execution User (in this case it is egoadmin)
```
[WMLA-server dlidata]# chown -R egoadmin:egoadmin pytorch-mnist/
[WMLA-server hymenopteradata]# pwd
/dlidata/pytorch-mnist
[WMLA-server hymenopteradata]# ls -lt
total 0
drwxr-x--- 4 egoadmin egoadmin 34 Jan  7 23:54 MNIST

```

5. Take note of the path name of this dataset, note that your path will likely be different that seen here.
```
/dlidata/pytorch-mnist
```


## Set up API end point and log on
[Back to top](#Contents)

In this section we set up the API endpoint which will be used in this notebook.

1. Source the environment

$EGO_TOP is the installation directory.  The default value is /opt/ibm/spectrumcomputing

```
. $EGO_TOP/profile.platform

```
2. Login

```
egosh user logon -u <wmla_user>
Logged on successfully

```

3. Retrieve Conductor Rest API Port

```
egosh client view |grep -A 3 ASCD_REST_BASE_URL_1
CLIENT NAME: ASCD_REST_BASE_URL_1
DESCRIPTION: http://<WMLA-server>:8280/platform/rest/

```

4.  Retrieve DLI (Deep Learning Impact) Rest API Port

```
egosh client view |grep -A 3 DLPD_REST_BASE_URL_1
CLIENT NAME: DLPD_REST_BASE_URL_1
DESCRIPTION: http://<WMLA-server>:9280/platform/rest/

```

5.  Note that the port numbers in your URL will depend on whether SSL has been enabled or not



## Submit job via API
[Back to top](#Contents)

Now we need to structure our API job submission. There are various elements to this process as seen in the diagram below. Note that **this** jupyter notebook is the one referred to below. A [static version](https://github.com/IBM/wmla-assets/raw/master/WMLA-learning-journey/shared-images/4_api_setup.png) is also available.

![code](https://github.com/IBM/wmla-assets/raw/master/WMLA-learning-journey/shared-images/4_api_setup.gif)


The following sections use the Watson ML Accelerator API to complete the various tasks required. 
We've given examples of a number of tasks but you should refer to the documentation at to see more details 
of what is possible and sample output you might expect.

- https://www.ibm.com/support/knowledgecenter/SSFHA8_1.2.1/cm/deeplearning.html
- https://www.ibm.com/support/knowledgecenter/SSZU2E_2.4.0/reference_s/api_references.html

In [2]:
import requests, json
import pandas as pd
import datetime
# from IPython.display import display

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
# plt.rcParams['figure.figsize'] = [24, 8.0]
#import seaborn as sns

pd.set_option('display.max_columns', 999)
pd.set_option('max_colwidth', 300)

import tarfile
import tempfile
import os


In [6]:
# Environment details:

model_path = '/Users/kelvinlui/Github/wmla-assets/dli-learning-path/datasets/pytorch-mnist-edt-model'

protocol = 'https'

#master_host = 'colonia05.platform'
master_host = 'haswell01.eng.platformlab.ibm.com'

dli_rest_port = '55557'
sc_rest_port = '8643'


sc_rest_url =  protocol+'://'+master_host+':'+sc_rest_port+'/platform/rest/conductor/v1'
dl_rest_url = protocol+'://'+master_host+':'+dli_rest_port+'/platform/rest/deeplearning/v1'

print (sc_rest_url)
print (dl_rest_url)
# User login details

wmla_user = 'Admin'
wmla_pwd = 'Admin'


myauth = (wmla_user, wmla_pwd)

# Spark instance group details
#sig_name = '**** ADD HERE ****'
#sigName = 'ClaimsDamageDetection-IG'
sigName = 'EDT-IG'

# REST call variables
headers = {'Accept': 'application/json'}


#startTuneUrl='%s://%s:%s/platform/rest/deeplearning/v1/hypersearch' % (protocol, master_host, dli_rest_port)
#sc_rest_url ='%s://%s:%d/platform/rest/conductor/v1' % (protocol, hostname, conductorport)

req = requests.Session()

https://haswell01.eng.platformlab.ibm.com:8643/platform/rest/conductor/v1
https://haswell01.eng.platformlab.ibm.com:55557/platform/rest/deeplearning/v1


In [15]:
import os
model_dir = f'./resnet-wmla' 
model_main = f'elastic-main.py'
model_callback = f'edtcallback.py'
model_elog = f'elog.py'

os.makedirs(model_dir, exist_ok=True)

In [16]:
%%writefile {model_dir}/{model_main}
#!/usr/bin/env python

from __future__ import print_function
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms, models
from callbacks import Callback
from fabric_model import FabricModel
import os


def get_max_worker():
    import argparse
    parser = argparse.ArgumentParser(description='EDT Example')
    parser.add_argument('--numWorker', type=int, default='16', help='input the max number ')
    parser.add_argument('--gpuPerWorker', type=int, default='1', help='input the path of initial weight file')
    args, _ = parser.parse_known_args()
    num_worker = args.numWorker * args.gpuPerWorker
    return num_worker

BATCH_SIZE_PER_DEVICE = 64
NUM_EPOCHS = 20
MAX_NUM_WORKERS = get_max_worker()

START_LEARNING_RATE = 0.4
LR_STEP_SIZE = 30
LR_GAMMA = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4

print("DATA_DIR: " + os.getenv("DATA_DIR"))
DATA_DIR = os.getenv("DATA_DIR")
TRAIN_DATA = DATA_DIR + "/cifar10"
TEST_DATA = DATA_DIR + "/cifar10"

class LRScheduleCallback(Callback):
    def __init__(self, step_size, gamma):
        super(LRScheduleCallback, self).__init__()
        self.step_size = step_size
        self.gamma = gamma

    def on_epoch_begin(self, epoch):
        if (epoch != 0) and (epoch % self.step_size == 0):
            for param_group in self.params['optimizer'].param_groups:
                param_group['lr'] *= self.gamma

        print("LRScheduleCallback epoch={}, learning_rate={}".format(epoch,
              self.params['optimizer'].param_groups[0]['lr']))

def getDatasets():
    transform_train = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    return (torchvision.datasets.CIFAR10(root=TRAIN_DATA, train=True, download=True, transform=transform_train),
            torchvision.datasets.CIFAR10(root=TEST_DATA, train=False, download=True, transform=transform_test))

def main(model_type):
    print('==> Building model..' + str(model_type))
    model = models.__dict__[model_type]()

    optimizer = optim.SGD(model.parameters(), lr=START_LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    edt_m = FabricModel(model, getDatasets, F.cross_entropy, optimizer, user_callback=[LRScheduleCallback(LR_STEP_SIZE, LR_GAMMA)])
    print('==> epochs:' + str(NUM_EPOCHS) + ', batchsize:' + str(BATCH_SIZE_PER_DEVICE) + ', engines_number:' + str(MAX_NUM_WORKERS))
    edt_m.train(NUM_EPOCHS, BATCH_SIZE_PER_DEVICE, MAX_NUM_WORKERS, num_dataloader_threads=4, validation_freq=10, checkpoint_freq=0)

if __name__ == '__main__':
    main("resnet50")


Writing ./resnet-wmla/elastic-main.py


In [17]:
%%writefile {model_dir}/{model_callback}
#! /usr/bin/env python

from __future__ import print_function

import sys
import os

from callbacks import LoggerCallback
from emetrics import EMetrics
from elog import ELog

class EDTLoggerCallback(LoggerCallback):
    def __init__(self):
        self.gs =0

    def log_train_metrics(self, loss, acc, completed_batch,  worker=0):
        acc = acc/100.0
        self.gs += 1
        with EMetrics.open() as em:
            em.record(EMetrics.TEST_GROUP,completed_batch,{'loss': loss, 'accuracy': acc})
        with ELog.open() as log:
            log.recordTrain("Train", completed_batch, self.gs, loss, acc, worker)

    def log_test_metrics(self, loss, acc, completed_batch, worker=0):
        acc = acc/100.0
        with ELog.open() as log:
            log.recordTest("Test", loss, acc, worker)

Writing ./resnet-wmla/edtcallback.py


In [18]:
%%writefile {model_dir}/{model_elog}
import time
import os

class ELog(object):

    def __init__(self,subId,f):
        if "TRAINING_ID" in os.environ:
            self.trainingId = os.environ["TRAINING_ID"]
        elif "DLI_EXECID" in os.environ:
            self.trainingId = os.environ["DLI_EXECID"]
        else:
            self.trainingId = ""
        self.subId = subId
        self.f = f

    def __enter__(self):
        return self

    def __exit__(self, type, value, tb):
        self.close()

    @staticmethod
    def open(subId=None):
        if "LOG_DIR" in os.environ:
            folder = os.environ["LOG_DIR"]
        elif "JOB_STATE_DIR" in os.environ:
            folder = os.path.join(os.environ["JOB_STATE_DIR"],"logs")
        else:
            folder = "/tmp"

        if subId is not None:
            folder = os.path.join(folder, subId)

        if not os.path.exists(folder):
            os.makedirs(folder)

        f = open(os.path.join(folder, "stdout"), "a")
        return ELog(subId,f)

    def recordText(self,text):
        timestr = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        timestr = "["+ timestr + "]"
        if self.f:
            self.f.write(timestr + " " + text + "\n")
            self.f.flush()

    def recordTrain(self,title,iteration,global_steps,loss,accuracy,worker):
        text = title
        text = text + ",	Timestamp: " + str(int(round(time.time() * 1000)))
        text = text + ",	Global steps: " + str(global_steps)
        text = text + ",	Iteration: " + str(iteration)
        text = text + ",	Loss: " + str(float('%.5f' % loss) )
        text = text + ",	Accuracy: " + str(float('%.5f' % accuracy) )
        self.recordText(text)

    def recordTest(self,title,loss,accuracy,worker):
        text = title
        text = text + ",	Timestamp: " + str(int(round(time.time() * 1000)))
        text = text + ",	Loss: " + str(float('%.5f' % loss) )
        text = text + ",	Accuracy: " + str(float('%.5f' % accuracy) )
        self.recordText(text)

    def close(self):
        if self.f:
            self.f.close()

Writing ./resnet-wmla/elog.py


In [20]:
!pwd

/Users/kelvinlui/Github/wmla-assets/dli-learning-path/tutorials-notebook


In [21]:
MODEL_DIR_SUFFIX = ".modelDir.tar"
tempFile = tempfile.mktemp(MODEL_DIR_SUFFIX)

#DATA_DIR=os.getenv('DATA_DIR')
make_tarfile(tempFile, './resnet-wmla')
files = {'file': open(tempFile, 'rb')}

#sig_name='fakesig'
#--msd-task12n-device-type cpu --conda-env-name dlipy36-cpu
args = '--exec-start edtPyTorch --cs-datastore-meta type=fs \
                     --numWorker 1 \
                     --model-main elastic-main.py --model-dir resnet-wmla'

### Package model files for training
Package the updated model files into a tar file ending with `.modelDir.tar`

In [12]:
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))


MODEL_DIR_SUFFIX = ".modelDir.tar"
tempFile = tempfile.mktemp(MODEL_DIR_SUFFIX)
make_tarfile(tempFile, '/project_data/data_asset/pytorch-resnet/resnet-wmla')
print(" tempFile: " + tempFile)
files = {'file': open(tempFile, 'rb')}

FileNotFoundError: [Errno 2] No such file or directory: '/project_data/data_asset/pytorch-resnet/resnet-wmla'

### Log on


Obtain login session tokens to be used for session authentication within the RESTful API. Tokens are valid for 8 hours.

In [13]:
r = requests.get(sc_rest_url+'/auth/logon', verify=False, auth=myauth, headers=headers) 

if r.ok:
    print ('\nLogon succeeded')
    
else: 
    print('\nLogon failed with code={}, {}'. format(r.status_code, r.content))


Logon succeeded


/Users/kelvinlui/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


### Check DL Frameworks details

Check what framework plugins are available and see example execution commands.  In this demonstration we will use **edtPyTorch**

In [14]:
r = requests.get(dl_rest_url+'/execs/frameworks', auth=myauth, headers=headers, verify=False).json()
# Using the raw json, easier to see the examples given
print(json.dumps(r, indent=4))

[
    {
        "name": "disttensorflow",
        "description": "",
        "desc": [
            "Distributed TensorFlow",
            "Instead of passing parameters such as ps_hosts, worker_hosts, specify --numPs",
            "as in example below. Parameter servers (ps) and worker hosts will be allocated",
            "dynamically.",
            "Use --gpuPerWorker flag to specify number of GPUs per worker.",
            "The maximum number of worker is 2.",
            "The maximum number of GPUs per worker is 2.",
            "Examples:",
            "$ python dlicmd.py --exec-start disttensorflow <connection-options> --ig <ig> --cs-datastore-meta type=fs,data_path=mnist --model-main mnist.py --numPs 1 --numWorker 1 --gpuPerWorker 1",
            "$ python dlicmd.py --exec-start disttensorflow <connection-options> --ig <ig> --cs-datastore-meta type=fs,data_path=mnist --model-main mnist.py --numPs 1 --numWorker 2 --gpuPerWorker 1"
        ],
        "distributeStrategy": "MultiWor

/Users/kelvinlui/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


### Arguments for API call
Equivalent of flags used if running command directly on WMLA CLI, including:


In [22]:
framework_name = 'edtPyTorch' # DL Framework to use, from list given above
dataset_location = 'pytorch-mnist' # relative path of your data set under $DLI_DATA_FS
local_dir_containing_your_code = 'resnet-wmla'
number_of_GPU = '4' # number of GPUs for elastic distribution
name_of_your_code_file = 'elastic-main.py' # Main model file as opened locally above


args = '--exec-start {} \
        --cs-datastore-meta type=fs,data_path={} \
        --model-dir {} \
        --edt-options maxWorkers={} \
        --model-main {} \
        --epochs 5  \
        '.format(framework_name, dataset_location, local_dir_containing_your_code, number_of_GPU, name_of_your_code_file)

print ("args: " + args)

args: --exec-start edtPyTorch         --cs-datastore-meta type=fs,data_path=pytorch-mnist         --model-dir resnet-wmla         --edt-options maxWorkers=4         --model-main elastic-main.py         --epochs 5          


### Submit Job

In [23]:
r = requests.post(dl_rest_url+'/execs?sigName='+sigName+'&args='+args, files=files,
                  auth=myauth, headers=headers, verify=False)

if r.ok:
    exec_id = r.json()['id']
    sig_id = r.json()['sigId']
    driver_id = r.json()['submissionId']
    print ('\nModel submitted successfully \Driver ID: {}'.format(driver_id))
    print ('Exec ID: {}'.format(exec_id))
    print ('SIG ID: {}'.format(sig_id))
else: 
    print('\nModel submission failed with code={}, {}'. format(r.status_code, r.content))

/Users/kelvinlui/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



Model submission failed with code=400, b'Error 400: Instance group name dli-edt not found.\n'


## Monitor running job
[Back to top](#Contents)

Once the job is submitted successfully we can monitor the running job. 



In [ ]:
# Check status of all RUNNING jobs in SIG (rerun cell to refresh)

monitor = []
monitor_output = []

r = requests.get(sc_rest_url+'/instances/'+sig_id+'/applications?state=RUNNING', 
                auth=myauth, headers=headers, verify=False).json()


       
if (len(r) == 0):
    print ('No jobs running')
    
else:
    
    # Filter out the relevant information
    monitor.append([(
        job['driver']['id'],
        job['driver']['state'],
        job['apprunduration'],
        job['gpuslots'],
        job['gpumemused']['total'],
        job['gpudevutil']['total'],
    ) for job in r])

    monitor_output = pd.DataFrame([item for monitor in monitor for item in monitor])
    monitor_output.columns = [
        'Driver ID', 
        'State', 
        'Run duration (mins)',
        'GPU slots',
        'Total GPU memory used',
        'Total GPU utilsation (%) ',
    ]
    
    for job in r:
        executors = job['executors']
        

monitor_output

## Retrieve output and saved models
[Back to top](#Contents)

After the job completes then we can retrieve the output, logs and saved models. 



### Output:  Retrieve Training Metric



In [ ]:
r = requests.get(dl_rest_url+'/execs/'+exec_id+'/log', auth=myauth, headers=headers, verify=False).json()

from io import StringIO

cols = ['timestamp','global_steps','iteration','loss','accuracy']
final_data = pd.read_csv(StringIO(r.replace(':',',')), 
                 usecols=[4,6,8,10,12], 
                 names=cols)
final_data['timestamp2'] = final_data.timestamp.apply(pd.to_datetime, unit='ms')
final_data.head(10)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
# plt.rcParams['figure.figsize'] = [24, 8.0]
import seaborn as sns

fig, axes = plt.subplots(2, 2, figsize = (15,10))


sns.lineplot(final_data.timestamp2,final_data.global_steps, color="g", ax=axes[0,0])
axes[0,0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
axes[0,0].set_xlabel("Time")

sns.lineplot(final_data.global_steps,final_data.loss, color="r", ax=axes[0,1])
sns.lineplot(final_data.global_steps,final_data.accuracy, color="b", ax=axes[1,0])

axes[1,1].axes.get_xaxis().set_visible(False)
axes[1,1].axes.get_yaxis().set_visible(False)
axes[1,1].text(0.05, 0.8, 'ID: '+exec_id, size=14)
axes[1,1].text(0.05, 0.65, 'SIG: '+sig_name, size=14)
#axes[1,1].text(0.05, 0.55, 'Status: '+status, size=14)

plt.show()

### Save Model

In [ ]:
# Get model from training job - downloads zip file (with progress bar) of saved model to directory local to this notebook
# (note that you need to save model in your code using the environment variable for location)

import requests, zipfile, io
#from tqdm.notebook import tqdm
import tqdm


r = requests.get(dl_rest_url+'/execs/'+'exec_id'+'/result', auth=myauth, stream=True)

total_size = int(r.headers.get('Content-Disposition').split('size=')[1])
block_size = 1024 #1 Kibibyte
t=tqdm(total=total_size, unit='iB', unit_scale=True)

with open('model.zip', 'wb') as f:
    for data in r.iter_content(block_size):
        t.update(len(data))
        f.write(data)
t.close()

## Debugging any issues
[Back to top](#Contents)

In the case where you have issues during the process detailed above, there are a number of detailed logs that you can view to understand what is happening on the WMLA cluster.

WMLA leverages Spark architecture for distributing Deep Learning/Machine Learning jobs.  In Spark,  when an item of processing has to be done, there is a “driver” process that is in charge of taking the user’s code and converting it into a set of multiple tasks. There are also “executor” processes, each operating on a separate node in the cluster, that are in charge of running the tasks, as delegated by the driver.

You can monitor Deep Learning/Machine Learning application activity,  performance and resource usage in Driver Log & Executor Log.
- Driver Log captures issues related to dependencies and environment variable,  for example,  missing dataset or invalid execution parameter flags.
- Executor Log records Deep Learning/Machine Learning training process. 



### Retrieve Training Driver Stdout Log

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

retrieve_stdout_url =  dl_rest_url + '/execs/'+exec_id+'/log?logType=' + 'outlog'
print ("retrieve_stdout_url: " + str(retrieve_stdout_url))
res = requests.get(retrieve_stdout_url, auth=('dse_user', 'cpd4ever'), headers={'Accept': 'application/json'}, verify=False)

pp.pprint(res.json())


In [ ]:
retrieve_stderr_url =  dl_rest_url + '/execs/'+exec_id+'/log?logType=' + 'errlog'
print ("retrieve_stderr_url: " + str(retrieve_stderr_url))
res = requests.get(retrieve_stderr_url, auth=myauth, headers={'Accept': 'application/octet-stream'}, verify=False)
pp = pprint.PrettyPrinter(indent=2)
print(res.json())

In [ ]:
# Get Spectrum Conductor logs for training run - shows various information including environment variables

r = requests.get(sc_rest_url+'/instances/'+sig_id+'/applications/'+driver_id+'/logs/stdout/download',
                 auth=myauth, headers={'Accept': 'application/octet-stream'}, verify=False)

print(r.text)


Note that the output from the training can be found in the `$DLI_WORK_DIR` referenced in this log, in the directory *under* `batchworkdir`. The structure of the files contained in this directory (which you will access via the API) are the following.

```
$ tree -h
.
├── [   6]  checkpoint
├── [ 247]  log
│   ├── [   6]  0-97eb84d4-6e4b-4bb7-95e0-fc7bfda461dc.<wmla_server>
│   ├── [   6]  1-a111dd6d-c406-48f2-89ce-1ef526d5b34b.<wmla_server>
│   └── [  50]  driver-20200227104231-0007-3655c5b5-5d81-43ac-a8c6-c243635f60df.<wmla_server>
│       ├── [ 19K]  evaluation-metrics.txt
│       └── [7.1K]  stdout
├── [  19]  model
│   └── [ 214]  train
│       ├── [4.7K]  model_epoch_10_optimizer_state.pth
│       ├── [ 43M]  model_epoch_10.pth
│       ├── [4.7K]  model_epoch_5_optimizer_state.pth
│       ├── [ 43M]  model_epoch_5.pth
│       ├── [4.7K]  model_epoch_final_optimizer_state.pth
│       └── [ 43M]  model_epoch_final.pth
├── [  25]  _submitted_code
│   └── [ 133]  pytorch_edt
│       ├── [1.6K]  edtcallback.py
│       ├── [2.0K]  elog.py
│       ├── [4.1K]  emetrics.py
│       ├── [  67]  __pycache__
│       │   ├── [2.0K]  edtcallback.cpython-36.pyc
│       │   └── [2.4K]  elog.cpython-36.pyc
│       ├── [3.1K]  pytorch_mnist_EDT.py
│       └── [4.4K]  pytorch_mnist.py
└── [2.5K]  val_dict_list.json
```

### Retrieve Training Driver Stderr Log

In [ ]:
# shows various information including environment variables
import pprint
r = requests.get(sc_rest_url+'/instances/'+sig_id+'/applications/'+driver_id+'/logs/stderr/download',
                 auth=myauth, headers={'Accept': 'application/octet-stream'}, verify=False)

print(r.text)

### Retrieve Training Executor Log
#### Retrieve Executor ID
- The deep learning training log per GPU is written in executor log
- Execute following code to retrieve list of executor ID

In [ ]:
for key in executors:    
    print ('executors: ' + key['id'])

### Retrieve Executor Stdout log
- set the parameter executor_id

In [ ]:
executor_id = '0-ee39e183-f4af-4b72-9bd2-89e09cbef18d'

r = requests.get(sc_rest_url+'/instances/'+sig_id+'/applications/'+driver_id +'/'+executor_id+'/logs/stdout/download',
                 auth=myauth, headers={'Accept': 'application/octet-stream'}, verify=False)

print(r.text)



### Retrieve Executor Stderr log
- set the parameter executor_id

In [ ]:
executor_id = '0-ee39e183-f4af-4b72-9bd2-89e09cbef18d'

r = requests.get(sc_rest_url+'/instances/'+sig_id+'/applications/'+driver_id +'/'+executor_id+'/logs/stderr/download',
                 auth=myauth, headers={'Accept': 'application/octet-stream'}, verify=False)

print(r.text)

## Further information and useful links
[Back to top](#Contents)

**WMLA Introductory videos:**
- WMLA overview video (6 mins): http://ibm.biz/WMLA-video
- WMLA getting started (3 mins): http://ibm.biz/WMLA-start
- Overview of adapting your code for Elastic Distributed Training via API: [video](https://youtu.be/RnZtYNX6meM) | [PDF](docs/wmla_api_pieces.pdf) (screenshot below)

**Further WMLA information & documentation**
- [Power Developer Portal (WMLCE frameworks information)](https://developer.ibm.com/linuxonpower/deep-learning-powerai/releases/)
- WMLA tutorials: http://ibm.biz/WMLA-blog
- Knowledge Centre links:
  - [IBM Watson Machine Learning Accelerator](https://www.ibm.com/support/knowledgecenter/SSFHA8)
  - [IBM Spectrum Conductor](https://www.ibm.com/support/knowledgecenter/en/SSZU2E/product_welcome_conductorspark.html)
  - [IBM Spectrum Conductor Deep Learning Impact](https://www.ibm.com/support/knowledgecenter/SSWQ2D)

**Further Power Systems information**
- [AI on Power Systems Redbook (PDF)](https://www.redbooks.ibm.com/redbooks/pdfs/sg248435.pdf) (deep dive technical information)

## Appendix
[Back to top](#Contents)

#### This notebook requires Watson ML Accelerator 1.2.1 + Interim Fix 527174 & 536919 to run.
For details please refer to following link:
https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/wmla_fix_pack.html

#### This is version 1.0 and its content is copyright of IBM.   All rights reserved.   


